# **Wine Quality Prediction**

## Introduction

Wine quality is often judged by trained tasters, which can be subjective and time consuming. By applying machine learning to the chemical properties of wine, we aim to create a reliable and scalable system to classify wines into low, medium, or high quality. The model can make quality control easier for producers and help consumers find wines of better quality.

## Data Preparation: Merging & Cleaning Wine Data

We first merge red and white wine datasets, standardize the numeric features, and save the cleaned version as `cleaned_wine_data.csv` for use in Spark and ML models.


In [41]:
# Load libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [43]:
# Read red and white wine datasets
red_wine = pd.read_csv("winequality-red.csv", sep=';')
white_wine = pd.read_csv("winequality-white.csv", sep=';')

In [45]:
# Add numeric wine_type column: 0 for red, 1 for white
red_wine["wine_type"] = 0
white_wine["wine_type"] = 1

In [47]:
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0


In [49]:
white_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1


In [51]:
print(red_wine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
 12  wine_type             1599 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 162.5 KB
None


In [53]:
print(white_wine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
 12  wine_type             4898 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 497.6 KB
None


In [55]:
# Combine both datasets
combined_wine = pd.concat([red_wine, white_wine], ignore_index=True)

In [57]:
# Check for missing values (optional)
print("Missing values before cleaning:")
print(combined_wine.isnull().sum())

Missing values before cleaning:
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
wine_type               0
dtype: int64


In [59]:
# Standardize numerical features
features_to_scale = combined_wine.drop(columns=["quality", "wine_type"]).columns
scaler = StandardScaler()
scaled_features = scaler.fit_transform(combined_wine[features_to_scale])

scaled_df = pd.DataFrame(scaled_features, columns=features_to_scale)
scaled_df["quality"] = combined_wine["quality"]
scaled_df["wine_type"] = combined_wine["wine_type"]

# Round all numeric columns to 4 decimal places
scaled_df = scaled_df.round(4)

# Sort the DataFrame
scaled_df = scaled_df.sort_values(by="quality", ascending=True)

In [61]:
# Save cleaned data to CSV
scaled_df.to_csv("cleaned_wine_data.csv", index=False)
print("Data cleaning and preprocessing completed.")

Data cleaning and preprocessing completed.



## Analyze Cleaned Wine Data using Spark SQL

We now use Spark to analyze the cleaned wine dataset. This demonstrates working with large datasets using distributed computing tools.

We'll register the data as a temporary SQL table and run queries on:
- Average wine quality by type
- Distribution of alcohol content
- Statistical summary of features


In [65]:
!pip install pyspark

In [67]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("WineQualityAnalysis").getOrCreate()

# Load cleaned wine dataset
df_spark = spark.read.csv("cleaned_wine_data.csv", header=True, inferSchema=True)

# Register temp view for SQL queries
df_spark.createOrReplaceTempView("wine_data")

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/opt/anaconda3/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [32]:
# Query: Average Wine Quality By Type
print("Average Quality By Wine Type")
avg_quality_spark_df = spark.sql("""
    SELECT wine_type, ROUND(AVG(quality), 2) AS avg_quality
    FROM wine_data
    GROUP BY wine_type
""")
avg_quality_spark_df.show()

Average Quality By Wine Type


NameError: name 'spark' is not defined

In [18]:
# Convert to Pandas
avg_quality_df = avg_quality_spark_df.toPandas()

In [19]:
# Save to HDF5
avg_quality_df.to_hdf('wine_data.h5', key='avg_quality_by_type', mode='a')

On average, white wines scored slightly higher in quality (5.88) compared to red wines (5.63).
This could reflect differences in production, chemical balance, or taste profiles between the two wine types.

In [20]:
# Query: Top 5 wines with highest alcohol content
print("Top 5 Wines with Highest Alcohol Content")
top5_alcohol_spark_df = spark.sql("""
    SELECT alcohol, quality, wine_type
    FROM wine_data
    ORDER BY alcohol DESC
    LIMIT 5
""")
top5_alcohol_spark_df.show()

Top 5 Wines with Highest Alcohol Content
+-------+-------+---------+
|alcohol|quality|wine_type|
+-------+-------+---------+
| 3.6962|      5|        0|
| 3.1093|      7|        1|
| 2.9835|      7|        1|
| 2.9416|      6|        1|
| 2.9416|      6|        0|
+-------+-------+---------+



In [21]:
# Convert to Pandas
top5_alcohol_df = top5_alcohol_spark_df.toPandas()

In [22]:
# Save to HDF5
top5_alcohol_df.to_hdf('wine_data.h5', key='top5_alcohol_wines', mode='a')

The wines with the highest alcohol content were all white wines.
Interestingly, these wines didn't necessarily have the highest quality scores, suggesting that alcohol level alone doesn’t guarantee better quality.

In [23]:
# Query: Summary stats of key features
print("Feature Averages by Quality Level")
feature_avg_spark_df = spark.sql("""
    SELECT
        quality,
        ROUND(AVG(alcohol), 2) AS avg_alcohol,
        ROUND(AVG(sulphates), 2) AS avg_sulphates,
        ROUND(AVG(pH), 2) AS avg_pH,
        ROUND(AVG(density), 2) AS avg_density
    FROM wine_data
    GROUP BY quality
    ORDER BY quality DESC
""")
feature_avg_spark_df.show()

Feature Averages by Quality Level
+-------+-----------+-------------+------+-----------+
|quality|avg_alcohol|avg_sulphates|avg_pH|avg_density|
+-------+-----------+-------------+------+-----------+
|      9|       1.42|        -0.44|  0.56|      -1.08|
|      8|        1.0|        -0.13|  0.03|      -0.73|
|      7|       0.75|         0.11|  0.06|      -0.52|
|      6|       0.08|         0.01|   0.0|      -0.05|
|      5|      -0.55|        -0.03| -0.04|       0.38|
|      4|      -0.26|        -0.17|  0.08|       0.05|
|      3|      -0.23|        -0.17|  0.24|       0.35|
+-------+-----------+-------------+------+-----------+



In [24]:
# Convert to Pandas
feature_avg_df = feature_avg_spark_df.toPandas()

In [25]:
# Save to HDF5
feature_avg_df.to_hdf('wine_data.h5', key='feature_avg_by_quality', mode='a')

From the featured averages, we see that alcohol content is a strong positive indicator of wine quality. Higher rated wines tend to have significantly more alcohol. Density is negatively correlated, with high-quality wines being less dense. Sulphates and pH show weaker or inconsistent patterns, suggesting they’re less reliable as standalone predictors of wine quality.


## Build a Predictive Model


We'll convert wine quality scores into categories:
- **Low**: 3–5
- **Medium**: 6
- **High**: 7+

Then we train a Random Forest Classifier and evaluate it using accuracy, precision, recall, and F1-score.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

In [ ]:
# Load data
df = pd.read_csv("cleaned_wine_data.csv")

In [ ]:
# Convert quality into categories
def categorize_quality(q):
    if q <= 5:
        return "Low"
    elif q == 6:
        return "Medium"
    else:
        return "High"

df['quality_label'] = df['quality'].apply(categorize_quality)

# Encode categorical variables
df_encoded = pd.get_dummies(df, columns=['wine_type', 'quality_label'], drop_first=False)

# Prepare features and target
X = df_encoded.drop(columns=['quality', 'quality_label_High', 'quality_label_Low', 'quality_label_Medium'])
y = df['quality_label']

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Train Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# Predictions
y_pred = model.predict(X_test)

In [ ]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2%}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Model Accuracy: 89.46%

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1061
           1       0.80      0.57      0.67       239

    accuracy                           0.89      1300
   macro avg       0.85      0.77      0.80      1300
weighted avg       0.89      0.89      0.89      1300

Confusion Matrix:
[[1026   35]
 [ 102  137]]




## Model Optimization & Comparison

We compare the performance of three models:
- Random Forest
- Logistic Regression
- Support Vector Machine (SVM)

Each is trained with:
- Full feature set
- PCA-reduced feature set

We report:
- Accuracy
- Classification Report
- Confusion Matrix


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import numpy as np

# Reload data
df = pd.read_csv("cleaned_wine_data.csv")
df["high_quality"] = (df["quality"] >= 7).astype(int)

In [ ]:
# Features and target
X = df.drop(columns=["quality", "wine_type", "high_quality"])
y = df["high_quality"]

In [ ]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Initialize PCA
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
# Initialize models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC()
}

In [ ]:
# Store results
results = []

In [ ]:
for name, model in models.items():
    # Train and evaluate without PCA
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results.append((name, "Models With No PCA", acc))
    print(f"{name} - No PCA")
    print(f"Accuracy: {acc:.2%}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

Random Forest - No PCA
Accuracy: 89.46%
Confusion Matrix:
[[1026   35]
 [ 102  137]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      1061
           1       0.80      0.57      0.67       239

    accuracy                           0.89      1300
   macro avg       0.85      0.77      0.80      1300
weighted avg       0.89      0.89      0.89      1300

Logistic Regression - No PCA
Accuracy: 81.85%
Confusion Matrix:
[[1001   60]
 [ 176   63]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.94      0.89      1061
           1       0.51      0.26      0.35       239

    accuracy                           0.82      1300
   macro avg       0.68      0.60      0.62      1300
weighted avg       0.79      0.82      0.79      1300

SVM - No PCA
Accuracy: 84.38%
Confusion Matrix:
[[1029   32]
 [ 171   68]]
Classification Report:
              precision   

In [ ]:
for name, model in models.items():
    # Train and evaluate with PCA
    if name == "Random Forest":
        model_pca = RandomForestClassifier(n_estimators=100, random_state=42)
        model_pca.fit(X_train_pca, y_train)
        y_pred_pca = model_pca.predict(X_test_pca)
    else:
        # Create a new instance of the model for PCA
        model_pca = type(model)()
        model_pca.fit(X_train_pca, y_train)
        y_pred_pca = model_pca.predict(X_test_pca)
    acc_pca = accuracy_score(y_test, y_pred_pca)
    results.append((name, "Models With PCA", acc_pca))
    print(f"{name} - With PCA")
    print(f"Accuracy: {acc_pca:.2%}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_pca))
    print("Classification Report:")
    print(classification_report(y_test, y_pred_pca))

Random Forest - With PCA
Accuracy: 88.92%
Confusion Matrix:
[[1018   43]
 [ 101  138]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      1061
           1       0.76      0.58      0.66       239

    accuracy                           0.89      1300
   macro avg       0.84      0.77      0.80      1300
weighted avg       0.88      0.89      0.88      1300

Logistic Regression - With PCA
Accuracy: 81.85%
Confusion Matrix:
[[1031   30]
 [ 206   33]]
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.97      0.90      1061
           1       0.52      0.14      0.22       239

    accuracy                           0.82      1300
   macro avg       0.68      0.55      0.56      1300
weighted avg       0.78      0.82      0.77      1300

SVM - With PCA
Accuracy: 82.31%
Confusion Matrix:
[[1042   19]
 [ 211   28]]
Classification Report:
              precis

## Conclusion

In this supervised learning project, we classified wine samples into three quality categories:
- **0 = Low**, **1 = Medium**, **2 = High**

We applied multiple machine learning models with and without PCA, and evaluated their performance on test data.

---

### Model Comparison Summary

| Model                  | PCA Used | Notes |
|------------------------|----------|------------------------------|
| **Random Forest**      | No     | High accuracy, best overall model |
| **Random Forest**      | Yes    | Slightly reduced accuracy due to PCA |
| **SVM (Support Vector Machine)** | No     | Accurate but slower to train |
| **SVM**                | Yes    | Faster training, decent accuracy |
| **Logistic Regression**| No     | Basic, good baseline model |
| **Logistic Regression**| Yes    | Slight drop in accuracy with PCA |

---

### Model Insights:
- The Random Forest Model with no PCA performed best overall, achieving a strong classification accuracy of 89%, exceeding the 75% benchmark for predictive performance.

- PCA helped simplify the data, and all models still performed well. Random Forest stayed the most accurate at 88.9%, but Logistic Regression and SVM had trouble identifying high-quality wines after PCA.


### Summary

In this project, we set out to determine whether we could predict wine quality based on its chemical properties using machine learning techniques. The high model accuracy, especially with Random Forest reaching over 89%, shows that chemical properties like alcohol, sulphates, and density can be used to successfully predict wine quality. These results support our research question and confirm that machine learning is an effective tool for this task.

### What Was Achieved:
- Merged and cleaned two datasets red and white wine, then standardized them.
- Explored the cleaned data using SQL queries in Spark.
- Built and evaluated three predictive models: Random Forest, Logistic Regression, and SVM.
- We compared how well the models worked before and after using PCA to simplify the data.